## 0 Init

In [1]:
# Cancel the comment to install all the packages and libraries needed.
# ! pip install rasterio matplotlib rasterstats ipynbname imageio tqdm rasterstats
# ! pip install numpy==1.24.4CURR_PATH
# ! pip install libpysal
# ! pip install geopandas libpysal esda matplotlib
# ! pip install seaborn
# ! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Configuration
from pathlib import Path
import sys

CURR_PATH = Path().resolve()            # current file path
REPO_PATH = CURR_PATH.parent            # current repository path
DATA_PATH = REPO_PATH / "data"          # path for saving the data
DEMO_PATH = DATA_PATH / "demo-data"     # path for demo purpose 

SRC_PATH = REPO_PATH / "src"    # path for other sources
sys.path.append(str(SRC_PATH))  # add src to system path to import custom functions


## 1 Data Preparation: Temporal & Spatial Aggregation

- Initial data resolution:
    - Time: daily
    - Space: cell (mesh grid)

- Total four types of data with different aggregation level:
    1. Cell level & Daily Data (original parquet file)
    2. Cell level & Monthly Data
    3. Sub-region level & Daily Data
    4. Sub-region level & Monthly Data

### Addis Ababa

In [2]:
import numpy as np
import pandas as pd
# Read the Addis Ababa data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_addis_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [4]:
# type(full_df)
full_df.columns

Index(['geom_id', 'no2_mean', 'pop_sum_m', 'NTL_mean', 'road_len',
       'road_share', 'poi_count', 'poi_share', 'lu_industrial_area',
       'lu_industrial_share', 'lu_commercial_area', 'lu_commercial_share',
       'lu_residential_area', 'lu_residential_share', 'lu_retail_area',
       'lu_retail_share', 'lu_farmland_area', 'lu_farmland_share',
       'lu_farmyard_area', 'lu_farmyard_share', 'road_motorway_len',
       'road_trunk_len', 'road_primary_len', 'road_secondary_len',
       'road_tertiary_len', 'road_residential_len', 'fossil_pp_count',
       'geometry_x', 'date', 'no2_lag1', 'no2_neighbor_lag1', 'cloud_category',
       'LST_day_mean', 'landcover_2023', 'Shape_Leng', 'Shape_Area', 'ADM3_EN',
       'ADM3_PCODE', 'month'],
      dtype='object')

In [5]:
full_df.head()

,geom_id,no2_mean,pop_sum_m,NTL_mean,road_len,road_share,poi_count,poi_share,lu_industrial_area,lu_industrial_share,...,no2_lag1,no2_neighbor_lag1,cloud_category,LST_day_mean,landcover_2023,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,month
0,0,0.000051,969.68396,7.266073,5860.59401,0.000745,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
1,0,0.000050,969.68396,10.372897,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000051,0.000038,0.0,25.130,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
2,0,0.000047,969.68396,1.124154,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000050,0.000052,0.0,15.588,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
3,0,0.000047,969.68396,0.727840,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000047,0.000050,0.0,30.670,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
4,0,0.000058,969.68396,3.964316,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000047,0.000044,0.0,30.990,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01


First, generate the cell level, monthly data and save as csv file.

In [17]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'addis_monthly_cell.csv', index=False)

In [23]:
# monthly_cell.head()

Second, generate the sub-district level, daily data and save as csv file.

In [21]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'addis_daily_adm3.csv', index=False)

In [25]:
# daily_adm3.head()

Third, generate the sub-district level, monthly data and save as csv file.

In [26]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [28]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'addis_monthly_adm3.csv', index=False)

In [31]:
# monthly_adm3

### Baghdad

In [6]:
import numpy as np
import pandas as pd
# Read the data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_baghdad_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')
full_df = full_df.rename(columns={'temp_mean': 'LST_day_mean'})   # unify to LST_day_mean

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [8]:
full_df.columns

Index(['geom_id', 'no2_mean', 'pop_sum_m', 'NTL_mean', 'road_len',
       'road_share', 'poi_count', 'poi_share', 'lu_industrial_area',
       'lu_industrial_share', 'lu_commercial_area', 'lu_commercial_share',
       'lu_residential_area', 'lu_residential_share', 'lu_retail_area',
       'lu_retail_share', 'lu_farmland_area', 'lu_farmland_share',
       'lu_farmyard_area', 'lu_farmyard_share', 'road_motorway_len',
       'road_trunk_len', 'road_primary_len', 'road_secondary_len',
       'road_tertiary_len', 'road_residential_len', 'fossil_pp_count', 'TCI',
       'geometry_x', 'date', 'no2_lag1', 'no2_neighbor_lag1', 'cloud_category',
       'LST_day_mean', 'landcover_2023', 'Shape_Leng', 'Shape_Area', 'ADM3_EN',
       'ADM3_PCODE', 'month'],
      dtype='object')

First, generate the cell level, monthly data and save as csv file.

In [46]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_cell.csv', index=False)

In [ ]:
# monthly_cell.head()

Second, generate the sub-district level, daily data and save as csv file.

In [47]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_daily_adm3.csv', index=False)

In [49]:
daily_adm3

,date,ADM3_EN,LST_day_mean,no2_mean,no2_lag1,no2_neighbor_lag1,NTL_mean,TCI,pop_sum_m,road_len,...,lu_residential_area,lu_retail_area,lu_farmland_area,lu_farmyard_area,road_primary_len,road_motorway_len,road_trunk_len,road_secondary_len,road_tertiary_len,road_residential_len
0,2023-01-01,Abu Ghraib,12.420119,0.095418,0.000000,0.000000,16517.924950,2.796893e+06,5.111909e+05,2.712699e+06,...,2.436915e+07,0.0,3.297929e+07,8734.319149,72805.034518,134296.278767,0.000000,42553.429081,267884.154210,1.157555e+06
1,2023-01-01,Al-Fahama,12.771666,0.037629,0.000000,0.000000,7831.303788,3.661187e+07,1.019493e+06,1.771264e+06,...,3.925814e+07,0.0,1.072887e+06,0.000000,9374.771374,23504.337300,31207.293767,4882.810604,165507.886467,1.322054e+06
2,2023-01-01,Al-Jisr,13.350142,0.066570,0.000000,0.000000,6524.999007,5.499618e+06,2.537896e+05,6.784165e+05,...,9.367158e+06,0.0,1.893582e+07,0.000000,29253.292783,0.000000,31548.343818,14049.948014,52184.422521,3.835369e+05
3,2023-01-01,Al-Karrada Al-Sharqia,13.822724,0.043889,0.000000,0.000000,10284.304605,2.778583e+07,6.804654e+05,1.657440e+06,...,2.692580e+07,0.0,6.301526e+06,0.000000,86620.551798,32598.567966,3730.427599,40736.417691,118433.362443,8.482135e+05
4,2023-01-01,Al-Latifya,12.399308,0.072134,0.000000,0.000000,5038.490275,3.488123e+06,1.598628e+05,8.827485e+05,...,2.360183e+06,0.0,2.155193e+08,0.000000,7899.550833,58368.588112,39685.810012,40019.461397,59683.649226,2.846482e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,2024-12-31,Markaz Al-Karkh,13.878697,0.000000,0.021264,0.021154,8148.203962,2.332277e+05,1.772346e+05,7.375371e+05,...,1.005766e+07,0.0,0.000000e+00,0.000000,71372.128640,14098.977033,0.000000,50811.351119,31200.153262,2.529159e+05
15347,2024-12-31,Markaz Al-Mada'in,13.722056,0.000000,0.406985,0.404960,7029.665967,1.069760e+05,1.636998e+05,7.846149e+05,...,1.549490e+07,0.0,1.479552e+08,0.000000,22978.465260,0.000000,42400.937644,0.000000,42946.439662,4.669922e+05
15348,2024-12-31,Markaz Al-Mahmoudiya,13.583168,0.000000,0.034868,0.034775,1850.581081,1.531018e+05,1.072385e+05,4.749349e+05,...,8.522331e+03,0.0,3.965280e+07,0.000000,9762.885545,7638.868219,9296.516086,2541.127138,57374.549910,2.398848e+05
15349,2024-12-31,Markaz Al-Thawra,13.638146,0.000000,0.046052,0.046124,10100.558853,3.099584e+05,1.156935e+06,1.571223e+06,...,3.415564e+07,0.0,0.000000e+00,0.000000,109349.280838,12861.668398,0.000000,49075.556344,167318.888287,1.085157e+06


Third, generate the sub-district level, monthly data and save as csv file.

In [50]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [51]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_adm3.csv', index=False)

In [53]:
# monthly_adm3

## Models

### 2.1 Addis Ababa

### 2.2 Baghdad: Mobility Model

#### 0) Data Loading

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

baghdad_df = pd.read_csv(DATA_PATH / "temp" / 'baghdad_monthly_adm3.csv')

# Features and targets setup
features = [
    'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
    'NTL_mean', 'pop_sum_m', 'road_len',
    'poi_count', 'lu_industrial_area',
    'lu_commercial_area', 'lu_residential_area',
    'non_built_area'

    # ,'LST_day_mean','lu_retail_area',
    # 'lu_farmland_area',
    #    'lu_farmyard_area', 'road_primary_len', 'road_motorway_len',
    #    'road_trunk_len', 'road_secondary_len', 'road_tertiary_len',
    #    'road_residential_len', 'grassland_a', 'cropland_a', 'built_up_a',
    #    'snow_a', 'water_bod_a', 'wetland_a', 'sparse_veg_a', 'mangroves_a',
    #    'moss_a', 'unclassified_a'
]


#### 1.1) Models y=TCI/road_len

##### 1.1.1) Simple linear Regression

In [49]:
# Target definition
baghdad_df['y1'] = baghdad_df['TCI'] / baghdad_df['road_len']

# Train/test split
X = baghdad_df[features]
y = baghdad_df['y1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit & predict
lr = LinearRegression().fit(X_train, y_train)
y_pred_train = lr.predict(X_train)
y_pred_test  = lr.predict(X_test)

# Metrics
for label, y_true, y_pred in [
    ('TRAIN', y_train, y_pred_train),
    ('TEST',  y_test,  y_pred_test)
]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    print(f"Simple LR ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")

Simple LR (TRAIN): RMSE = 6.5752, R² = 0.5701
Simple LR (TEST): RMSE = 7.0133, R² = 0.5787


##### 1.1.2) Log-log Linear Regression

In [50]:
# Clone and avoid zeros: common practice is to add a small ε = half the minimum positive value
df_ll = baghdad_df.copy()
epsilon = df_ll[features + ['TCI', 'road_len']].replace(0, np.nan).min().min() / 2

# Log transformation
for col in features:
    df_ll[col] = np.log(df_ll[col].clip(lower=epsilon))
df_ll['y2'] = np.log((df_ll['TCI'] / df_ll['road_len']).clip(lower=epsilon))

# Split
X = df_ll[features]
y = df_ll['y2']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit & evaluate
lr_ll = LinearRegression().fit(X_train, y_train)
for label, X_, y_, model in [
    ('TRAIN', X_train, y_train, lr_ll),
    ('TEST',  X_test,  y_test,  lr_ll)
]:
    pred = model.predict(X_)
    rmse = np.sqrt(mean_squared_error(y_, pred))
    r2   = r2_score(y_, pred)
    print(f"Log–Log LR ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")


Log–Log LR (TRAIN): RMSE = 5.1990, R² = 0.1374
Log–Log LR (TEST): RMSE = 5.0356, R² = 0.0052


##### 1.1.3) Polynomial Dg3

In [51]:
# Degree‐3 polynomial expansion (no interactions)
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(baghdad_df[features])

# Train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_poly, baghdad_df['y1'], test_size=0.2, random_state=42
)

# Fit & evaluate
poly_lr = LinearRegression().fit(X_train, y_train)
for label, X_, y_, model in [
    ('TRAIN', X_train, y_train, poly_lr),
    ('TEST',  X_test,  y_test,  poly_lr)
]:
    pred = model.predict(X_)
    rmse = np.sqrt(mean_squared_error(y_, pred))
    r2   = r2_score(y_, pred)
    print(f"Poly LR (deg 3) ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")

Poly LR (deg 3) (TRAIN): RMSE = 4.3668, R² = 0.8104
Poly LR (deg 3) (TEST): RMSE = 6.5631, R² = 0.6310


#### 1.2) Models y=TCI

##### 1.2.1) Simple linear Regression

In [52]:
X = baghdad_df[features]
y = baghdad_df['TCI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr2 = LinearRegression().fit(X_train, y_train)
for label, y_true, y_pred in [
    ('TRAIN', y_train, lr2.predict(X_train)),
    ('TEST',  y_test,  lr2.predict(X_test))
]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    print(f"Simple LR (TCI) ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")

Simple LR (TCI) (TRAIN): RMSE = 10025610.9905, R² = 0.6511
Simple LR (TCI) (TEST): RMSE = 11332689.0372, R² = 0.7308


##### 1.2.2) Log-log Linear Regression

In [53]:
# Add ε and log‐transform
df_ll2 = baghdad_df.copy()
epsilon = df_ll2[features + ['TCI']].replace(0, np.nan).min().min() / 2
for col in features:
    df_ll2[col] = np.log(df_ll2[col].clip(lower=epsilon))
df_ll2['y5'] = np.log(df_ll2['TCI'].clip(lower=epsilon))

X = df_ll2[features]
y = df_ll2['y5']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_ll2 = LinearRegression().fit(X_train, y_train)
for label, X_, y_, model in [
    ('TRAIN', X_train, y_train, lr_ll2),
    ('TEST',  X_test,  y_test,  lr_ll2)
]:
    pred = model.predict(X_)
    rmse = np.sqrt(mean_squared_error(y_, pred))
    r2   = r2_score(y_, pred)
    print(f"Log–Log LR (TCI) ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")

Log–Log LR (TCI) (TRAIN): RMSE = 5.9274, R² = 0.1163
Log–Log LR (TCI) (TEST): RMSE = 5.7096, R² = -0.0072


##### 1.2.3) Polynomial Dg3

In [54]:
poly2 = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_poly2 = poly2.fit_transform(baghdad_df[features])

X_train, X_test, y_train, y_test = train_test_split(
    X_poly2, baghdad_df['TCI'], test_size=0.2, random_state=42
)

poly2_lr = LinearRegression().fit(X_train, y_train)
for label, X_, y_, model in [
    ('TRAIN', X_train, y_train, poly2_lr),
    ('TEST',  X_test,  y_test,  poly2_lr)
]:
    pred = model.predict(X_)
    rmse = np.sqrt(mean_squared_error(y_, pred))
    r2   = r2_score(y_, pred)
    print(f"Poly LR (TCI, deg 3) ({label}): RMSE = {rmse:.4f}, R² = {r2:.4f}")

Poly LR (TCI, deg 3) (TRAIN): RMSE = 6199889.6561, R² = 0.8666
Poly LR (TCI, deg 3) (TEST): RMSE = 14277190.1367, R² = 0.5727


#### Metrics models performance

RMSE and R2 for all 6 linear models

In [ ]:
# ============================================================
#  PERFORMANCE DASHBOARD – all six baseline models
# ============================================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# ------------------------------------------------------------
# 1. Scoring utility
# ------------------------------------------------------------
def evaluate_model(model, X_tr, X_te, y_tr, y_te):
    """Return RMSE_train, R2_train, RMSE_test, R2_test."""
    y_hat_tr = model.predict(X_tr)
    y_hat_te = model.predict(X_te)
    rmse_tr  = np.sqrt(mean_squared_error(y_tr, y_hat_tr))
    rmse_te  = np.sqrt(mean_squared_error(y_te, y_hat_te))
    r2_tr    = r2_score(y_tr, y_hat_tr)
    r2_te    = r2_score(y_te, y_hat_te)
    return rmse_tr, r2_tr, rmse_te, r2_te

# ------------------------------------------------------------
# 2. Regenerate splits & capture metrics
# ------------------------------------------------------------
results = []

# --- 1) Simple LR on TCI/road_len ------------------------------------------
X = baghdad_df[features]
y = baghdad_df['y1']                              # target already computed
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
results.append(
    ['LR', 'TCI/road_len', *evaluate_model(lr, X_tr, X_te, y_tr, y_te)]
)

# --- 2) Log–Log LR on TCI/road_len -----------------------------------------
X = df_ll[features]
y = df_ll['y2']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
results.append(
    ['Log–Log LR', 'TCI/road_len', *evaluate_model(lr_ll, X_tr, X_te, y_tr, y_te)]
)

# --- 3) Poly-deg-3 LR on TCI/road_len --------------------------------------
poly_tmp = PolynomialFeatures(degree=3, include_bias=False)
X_poly   = poly_tmp.fit_transform(baghdad_df[features])
X_tr, X_te, y_tr, y_te = train_test_split(X_poly, baghdad_df['y1'],
                                          test_size=0.2, random_state=42)
results.append(
    ['Poly LR (d=3)', 'TCI/road_len', *evaluate_model(poly_lr, X_tr, X_te, y_tr, y_te)]
)

# --- 4) Simple LR on raw TCI ------------------------------------------------
X = baghdad_df[features]
y = baghdad_df['TCI']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
results.append(
    ['LR', 'TCI', *evaluate_model(lr2, X_tr, X_te, y_tr, y_te)]
)

# --- 5) Log–Log LR on raw TCI ----------------------------------------------
X = df_ll2[features]
y = df_ll2['y5']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
results.append(
    ['Log–Log LR', 'TCI', *evaluate_model(lr_ll2, X_tr, X_te, y_tr, y_te)]
)

# --- 6) Poly-deg-3 LR on raw TCI -------------------------------------------
poly_tmp2 = PolynomialFeatures(degree=3, include_bias=False)
X_poly2   = poly_tmp2.fit_transform(baghdad_df[features])
X_tr, X_te, y_tr, y_te = train_test_split(X_poly2, baghdad_df['TCI'],
                                          test_size=0.2, random_state=42)
results.append(
    ['Poly LR (d=3)', 'TCI', *evaluate_model(poly2_lr, X_tr, X_te, y_tr, y_te)]
)

# ------------------------------------------------------------
# 3. Executive summary table
# ------------------------------------------------------------
df_perf = pd.DataFrame(
    results,
    columns=['Model', 'Target', 'RMSE_train', 'R2_train', 'RMSE_test', 'R2_test']
).set_index(['Model', 'Target'])

display(
    df_perf.style.format('{:.4f}')
          .set_caption("Baseline Linear-Family Models – Performance KPI Matrix")
)


,,RMSE_train,R2_train,RMSE_test,R2_test
Model,Target,,,,
LR,TCI/road_len,6.5752,0.5701,7.0133,0.5787
Log–Log LR,TCI/road_len,5.1990,0.1374,5.0356,0.0052
Poly LR (d=3),TCI/road_len,4.3668,0.8104,6.5631,0.6310
LR,TCI,10025610.9905,0.6511,11332689.0372,0.7308
Log–Log LR,TCI,5.9274,0.1163,5.7096,-0.0072
Poly LR (d=3),TCI,6199889.6561,0.8666,14277190.1367,0.5727


AIC, BIC and adjusted R2 for different models

In [76]:
def compute_ic(model, X_tr, y_tr):
    """Return (p, AIC, BIC, R2_train, R2_adj)."""
    y_hat = model.predict(X_tr)
    rss   = ((y_tr - y_hat) ** 2).sum()
    n     = len(y_tr)
    p     = np.count_nonzero(model.coef_) + 1
    aic   = n * np.log(rss / n) + 2 * p
    bic   = n * np.log(rss / n) + p * np.log(n)
    r2    = r2_score(y_tr, y_hat)
    r2_adj= 1 - (1 - r2)*(n - 1)/(n - p - 1)
    return p, aic, bic, r2, r2_adj

def compute_test_metrics(model, X_te, y_te):
    """Return (RMSE_test, R2_test)."""
    y_hat = model.predict(X_te)
    return np.sqrt(mean_squared_error(y_te, y_hat)), r2_score(y_te, y_hat)

# container
rows = []

# 1) OLS on TCI/road_len
y1 = baghdad_df['TCI'] / baghdad_df['road_len']
X1 = baghdad_df[features]
X_tr, X_te, y_tr, y_te = train_test_split(X1, y1, test_size=0.2, random_state=42)
rows.append(
    ['OLS (TCI/road_len)',
     *compute_ic(lr, X_tr, y_tr),
     *compute_test_metrics(lr, X_te, y_te)]
)

# 2) Log–Log OLS on TCI/road_len
X2, y2 = df_ll[features], df_ll['y2']
X_tr, X_te, y_tr, y_te = train_test_split(X2, y2, test_size=0.2, random_state=42)
rows.append(
    ['LogLog (TCI/road_len)',
     *compute_ic(lr_ll, X_tr, y_tr),
     *compute_test_metrics(lr_ll, X_te, y_te)]
)

# 3) Poly3 OLS on TCI/road_len
poly = PolynomialFeatures(degree=3, include_bias=False)
X3 = poly.fit_transform(baghdad_df[features])
y3 = baghdad_df['TCI'] / baghdad_df['road_len']
X_tr, X_te, y_tr, y_te = train_test_split(X3, y3, test_size=0.2, random_state=42)
rows.append(
    ['Poly3 (TCI/road_len)',
     *compute_ic(poly_lr, X_tr, y_tr),
     *compute_test_metrics(poly_lr, X_te, y_te)]
)

# 4) OLS on TCI
X4, y4 = baghdad_df[features], baghdad_df['TCI']
X_tr, X_te, y_tr, y_te = train_test_split(X4, y4, test_size=0.2, random_state=42)
rows.append(
    ['OLS (TCI)',
     *compute_ic(lr2, X_tr, y_tr),
     *compute_test_metrics(lr2, X_te, y_te)]
)

# 5) Log–Log OLS on TCI
X5, y5 = df_ll2[features], df_ll2['y5']
X_tr, X_te, y_tr, y_te = train_test_split(X5, y5, test_size=0.2, random_state=42)
rows.append(
    ['LogLog (TCI)',
     *compute_ic(lr_ll2, X_tr, y_tr),
     *compute_test_metrics(lr_ll2, X_te, y_te)]
)

# 6) Poly3 OLS on TCI
poly2 = PolynomialFeatures(degree=3, include_bias=False)
X6 = poly2.fit_transform(baghdad_df[features])
y6 = baghdad_df['TCI']
X_tr, X_te, y_tr, y_te = train_test_split(X6, y6, test_size=0.2, random_state=42)
rows.append(
    ['Poly3 (TCI)',
     *compute_ic(poly2_lr, X_tr, y_tr),
     *compute_test_metrics(poly2_lr, X_te, y_te)]
)

# build DataFrame
df_compare = pd.DataFrame(
    rows,
    columns=[
      'Model', '#params', 'AIC', 'BIC', 'R2_train', 'R2_adj',
      'RMSE_test', 'R2_test'
    ]
).set_index('Model')

# display
display(
    df_compare.style
      .format({
         '#params':'{:.0f}',
         'AIC':'{:.2f}','BIC':'{:.2f}',
         'R2_train':'{:.4f}','R2_adj':'{:.4f}',
         'RMSE_test':'{:.2f}','R2_test':'{:.4f}'
      })
      .set_caption("Complexity‐Penalized & Out‐of‐Sample Performance")
)

,#params,AIC,BIC,R2_train,R2_adj,RMSE_test,R2_test
Model,,,,,,,
OLS (TCI/road_len),12,1541.94,1589.93,0.5701,0.5568,7.01,0.5787
LogLog (TCI/road_len),12,1352.67,1400.66,0.1374,0.1109,5.04,0.0052
Poly3 (TCI/road_len),364,1916.06,3371.68,0.8104,-1.0061,6.56,0.6310
OLS (TCI),12,13017.25,13065.23,0.6511,0.6404,11332689.04,0.7308
LogLog (TCI),12,1458.35,1506.34,0.1163,0.0891,5.71,-0.0072
Poly3 (TCI),364,13333.87,14789.49,0.8666,-0.4115,14277190.14,0.5727


Across both target definitions, the simple OLS approaches (12 parameters, no transforms or high-order terms) consistently give you the best out-of-sample R² while keeping model complexity and information‐criteria penalties low.

The polynomial expansions over-fit, and the log–log specs under-fit—they never beat plain OLS on real-world generalisation.

#### Experimentation to improve performance

##### Simple Lasso

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning

# ── Suppress warnings ─────────────────────────────────────
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# ── (Re)define the full feature set & data split ─────────
features_full = [
    'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
    'NTL_mean', 'pop_sum_m', 'road_len',
    'poi_count', 'lu_industrial_area',
    'lu_commercial_area', 'lu_residential_area',
    'non_built_area', 'LST_day_mean', 'lu_retail_area',
    'lu_farmland_area', 'lu_farmyard_area',
    'road_primary_len', 'road_motorway_len',
    'road_trunk_len', 'road_secondary_len',
    'road_tertiary_len', 'road_residential_len',
    'grassland_a', 'cropland_a', 'built_up_a',
    'water_bod_a', 'wetland_a'
    #,'snow_a'
    # ,'sparse_veg_a', 'mangroves_a', 'moss_a',
    # 'unclassified_a'
]

X_full = baghdad_df[features_full]
y_full = baghdad_df['TCI']
X_tr, X_te, y_tr, y_te = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

# ── Pipeline & Fit ────────────────────────────────────────
lasso_pipeline = Pipeline([
    ('scaler',  StandardScaler()),
    ('lassocv', LassoCV(
        alphas=np.logspace(-4, 1, 50),
        cv=5, max_iter=20000, n_jobs=-1, random_state=42
    ))
])
lasso_pipeline.fit(X_tr, y_tr)

# ── Metrics ───────────────────────────────────────────────
y_tr_pred = lasso_pipeline.predict(X_tr)
y_te_pred = lasso_pipeline.predict(X_te)

rmse_tr = np.sqrt(mean_squared_error(y_tr, y_tr_pred))
r2_tr   = r2_score(y_tr, y_tr_pred)
rmse_te = np.sqrt(mean_squared_error(y_te, y_te_pred))
r2_te   = r2_score(y_te, y_te_pred)
alpha_opt = lasso_pipeline.named_steps['lassocv'].alpha_

# ── Clean output ──────────────────────────────────────────
print(f"Optimal α        : {alpha_opt:.6f}")
print(f"Train →  RMSE = {rmse_tr:.4f}, R² = {r2_tr:.4f}")
print(f"Test  →  RMSE = {rmse_te:.4f}, R² = {r2_te:.4f}")


Optimal α        : 0.000100
Train →  RMSE = 8528955.0585, R² = 0.7475
Test  →  RMSE = 9837093.1042, R² = 0.7972


Extract coefficients

In [65]:
# 1. Extract fitted components
scaler = lasso_pipeline.named_steps['scaler']
lasso  = lasso_pipeline.named_steps['lassocv']

# 2. Scaled-space coefficients
coef_scaled = lasso.coef_

# 3. Back-transform to original units
coef_original = coef_scaled / scaler.scale_
intercept_original = (
    lasso.intercept_
    - np.dot(scaler.mean_ / scaler.scale_, coef_scaled)
)

# 4. Assemble into DataFrame
coef_df = pd.DataFrame({
    'feature': features_full,
    'coefficient': coef_original
}).set_index('feature')

# Add the intercept
coef_df.loc['Intercept'] = intercept_original

# 5. Sort by magnitude for readability
coef_df['abs_coef'] = coef_df['coefficient'].abs()
coef_df = coef_df.sort_values('abs_coef', ascending=False).drop(columns='abs_coef')

# 6. Print to console
print(coef_df)

                       coefficient
feature                           
no2_lag1              4.757508e+08
no2_neighbor_lag1    -2.840682e+08
no2_mean             -1.338793e+08
Intercept             1.704146e+07
poi_count             9.290695e+05
LST_day_mean         -4.622741e+05
lu_retail_area        9.373272e+02
NTL_mean             -6.388567e+02
lu_farmyard_area     -5.730282e+02
road_motorway_len    -3.758904e+02
road_trunk_len       -2.648180e+02
road_primary_len     -2.073009e+02
road_len              5.797252e+01
road_residential_len -5.014573e+01
road_secondary_len   -4.675715e+01
road_tertiary_len    -4.370022e+01
wetland_a            -2.552886e+01
pop_sum_m             2.223345e+01
grassland_a          -3.277168e+00
water_bod_a           3.898104e-01
cropland_a           -3.261144e-01
non_built_area        2.877204e-01
lu_commercial_area    2.606185e-01
built_up_a           -8.912455e-02
lu_industrial_area   -8.243068e-02
lu_residential_area  -7.556240e-02
lu_farmland_area    

In [66]:
coef_df.to_csv(r'C:\Users\Luis.ParraMorales\OneDrive - Arup\Documents\coef.csv',index=False)

##### LassoCV on degree-2 polynomials with pairwise interactions

In [69]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from sklearn.exceptions import ConvergenceWarning

# ── suppress warnings ─────────────────────────────────────
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# ── re-split train/test ──────────────────────────────────
X = baghdad_df[features_full]
y = baghdad_df['TCI']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

# ── pipeline skeleton ────────────────────────────────────
pipe = Pipeline([
    ('poly',  PolynomialFeatures(include_bias=False)),
    ('scale', StandardScaler()),
    ('lasso', Lasso(max_iter=20000, random_state=42))
])

# ── hyperparameter grid ──────────────────────────────────
param_grid = {
    'poly__degree': [1, 2, 3],
    'poly__interaction_only': [True, False],
    'lasso__alpha': np.logspace(-4, 1, 20)
}

# ── grid-search with train & val scoring ─────────────────
gs = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring='r2',
    return_train_score=True,
    n_jobs=-1
)
gs.fit(X_tr, y_tr)

# ── assemble CV results & compromise metric ──────────────
cv_df = pd.DataFrame(gs.cv_results_)[[
    'param_poly__degree',
    'param_poly__interaction_only',
    'param_lasso__alpha',
    'mean_train_score',
    'mean_test_score'
]].rename(columns={
    'param_poly__degree':'degree',
    'param_poly__interaction_only':'interactions_only',
    'param_lasso__alpha':'alpha',
    'mean_train_score':'r2_train_cv',
    'mean_test_score':'r2_val_cv'
})
cv_df['r2_min'] = cv_df[['r2_train_cv','r2_val_cv']].min(axis=1)

best_idx   = cv_df['r2_min'].idxmax()
best_cfg   = cv_df.loc[best_idx]
best_deg   = best_cfg['degree']
best_inter = best_cfg['interactions_only']
best_alpha = best_cfg['alpha']

# ── refit the compromise model ───────────────────────────
best_pipe = Pipeline([
    ('poly',  PolynomialFeatures(degree=int(best_deg),
                                 interaction_only=bool(best_inter),
                                 include_bias=False)),
    ('scale', StandardScaler()),
    ('lasso', Lasso(alpha=best_alpha,
                    max_iter=20000,
                    random_state=42))
])
best_pipe.fit(X_tr, y_tr)

# ── final evaluation ─────────────────────────────────────
y_tr_hat = best_pipe.predict(X_tr)
y_te_hat = best_pipe.predict(X_te)

rmse_tr = np.sqrt(mean_squared_error(y_tr, y_tr_hat))
r2_tr   = r2_score(y_tr, y_tr_hat)
rmse_te = np.sqrt(mean_squared_error(y_te, y_te_hat))
r2_te   = r2_score(y_te, y_te_hat)

print("▶︎ Best configuration:")
print(f"    • degree = {best_deg}")
print(f"    • interactions_only = {best_inter}")
print(f"    • α = {best_alpha:.6f}")
print("▶︎ Cross-val compromise R² (train/val) = "
      f"{best_cfg.r2_train_cv:.4f} / {best_cfg.r2_val_cv:.4f}")
print("▶︎ Final performance:")
print(f"    • Train → RMSE = {rmse_tr:.4f}, R² = {r2_tr:.4f}")
print(f"    • Test  → RMSE = {rmse_te:.4f}, R² = {r2_te:.4f}")



▶︎ Best configuration:
    • degree = 1
    • interactions_only = True
    • α = 0.000100
▶︎ Cross-val compromise R² (train/val) = 0.7510 / 0.6698
▶︎ Final performance:
    • Train → RMSE = 8528955.0585, R² = 0.7475
    • Test  → RMSE = 9837093.1042, R² = 0.7972


##### Random Forest Experimentation

In [71]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import shap, warnings, scipy.stats as st
warnings.filterwarnings('ignore')

# ── 1) Raw-unit feature matrix & target ────────────────────────────────
X_rf = baghdad_df[features_full].copy()
y_rf = baghdad_df['TCI'].copy()

X_tr_rf, X_te_rf, y_tr_rf, y_te_rf = train_test_split(
    X_rf, y_rf, test_size=0.2, random_state=42
)

# ── 2) Randomised hyper-tuning (20 draws) ──────────────────────────────
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_dist = {
    'n_estimators'    : st.randint(400, 1001),        # 400–1000 trees
    'max_depth'       : [None, 20, 40],
    'min_samples_leaf': [1, 3, 5],
    'max_features'    : ['sqrt', 0.7, 0.9]
}

rs = RandomizedSearchCV(
    rf,
    param_distributions = param_dist,
    n_iter      = 20,
    cv          = 5,
    scoring     = 'r2',
    return_train_score = True,
    n_jobs      = -1,
    random_state= 42
)
rs.fit(X_tr_rf, y_tr_rf)
best_rf = rs.best_estimator_

# ── 3) Evaluation ───────────────────────────────────────────────────────
for label, X_, y_ in [('TRAIN', X_tr_rf, y_tr_rf),
                      ('TEST',  X_te_rf, y_te_rf)]:
    pred = best_rf.predict(X_)
    rmse = np.sqrt(mean_squared_error(y_, pred))
    r2   = r2_score(y_, pred)
    print(f"RF ({label}) → RMSE = {rmse:.4f}, R² = {r2:.4f}")

print("Selected RF params:", rs.best_params_)

# ── 4) SHAP global importances ─────────────────────────────────────────
explainer = shap.TreeExplainer(best_rf)
shap_vals = explainer.shap_values(X_te_rf, check_additivity=False)
shap_df   = pd.Series(np.abs(shap_vals).mean(axis=0),
                      index=X_rf.columns,
                      name='mean|SHAP|').sort_values(ascending=False)

print("\nTop-10 SHAP drivers:")
print(shap_df.head(10).to_string(float_format='%.4f'))

# ── 5) Elasticity approximation (raw units) ────────────────────────────
def elasticity(model, x_row, feature, delta=0.01):
    """
    %Δy / %Δx  using finite difference on raw-unit model.
    """
    x_up = x_row.copy()
    bump = x_up[feature] * delta if x_up[feature] != 0 else delta
    x_up[feature] += bump
    y0 = model.predict(x_row.values.reshape(1, -1))[0]
    y1 = model.predict(x_up.values.reshape(1, -1))[0]
    if y0 == 0:            # guard against div-by-zero
        return np.nan
    return ((y1 - y0) / y0) / delta   # elasticity formula

sample = X_te_rf.iloc[0]
elas = {f: elasticity(best_rf, sample, f) for f in X_rf.columns}
elas_df = pd.Series(elas, name='elasticity').sort_values(
              key=lambda s: s.abs(), ascending=False)

print("\nLocal elasticities for one test sample (top-10):")
print(elas_df.head(10).to_string(float_format='%.4f'))


RF (TRAIN) → RMSE = 5791056.7935, R² = 0.8836
RF (TEST) → RMSE = 10372546.1159, R² = 0.7745
Selected RF params: {'max_depth': None, 'max_features': 0.9, 'min_samples_leaf': 5, 'n_estimators': 605}

Top-10 SHAP drivers:
road_primary_len       3846821.8062
poi_count              2325672.0367
non_built_area         1744155.0331
cropland_a             1713642.4255
LST_day_mean           1264173.3052
road_len               1035734.7067
road_residential_len    769925.1754
lu_residential_area     764448.9765
pop_sum_m               753231.9807
road_motorway_len       668670.4499

Local elasticities for one test sample (top-10):
LST_day_mean        -0.9786
built_up_a          -0.7177
wetland_a           -0.5245
water_bod_a          0.1236
grassland_a          0.1188
NTL_mean            -0.0563
cropland_a          -0.0375
road_trunk_len      -0.0000
pop_sum_m           -0.0000
road_motorway_len    0.0000


Elasticities

In [72]:
# %% [markdown]
# ##### Global elasticity distribution (RF, raw units)

# %%
import numpy as np
import pandas as pd

def elasticity_matrix(model, X, delta=0.01):
    """
    Vectorised elasticity for an entire sample matrix X.
    Returns: array (n_samples, n_features)
    ε_ij = ((f(x_i⊕δ_j) – f(x_i)) / f(x_i)) / δ
    """
    y_base = model.predict(X)
    X_bump = X.copy()
    elas = np.empty_like(X.values, dtype=float)

    for j, col in enumerate(X.columns):
        bump = X[col].values * delta
        bump[bump == 0] = delta        # guard zeros
        X_bump[col] = X[col] + bump
        y_bump = model.predict(X_bump)
        elas[:, j] = ((y_bump - y_base) / y_base) / delta
        X_bump[col] = X[col]           # restore

    return elas

# 1) Compute matrix on the whole test fold
E = elasticity_matrix(best_rf, X_te_rf, delta=0.01)   # shape (n_test, n_feat)

# 2) Wrap in DataFrame
elas_df = pd.DataFrame(E, columns=X_te_rf.columns)

# 3) Aggregate statistics
summary = (elas_df
           .agg(['mean','median',lambda s: s.quantile(0.1),lambda s: s.quantile(0.9)])
           .T.rename(columns={'<lambda_0>':'q10','<lambda_1>':'q90'}))

# 4) Rank by |mean|
summary['abs_mean'] = summary['mean'].abs()
summary = summary.sort_values('abs_mean', ascending=False).drop(columns='abs_mean')

print("Global elasticity summary (mean, median, 10–90 % deciles)\n")
print(summary.head(10).to_string(float_format='{:.4f}'.format))


Global elasticity summary (mean, median, 10–90 % deciles)

                      mean  median  <lambda>  <lambda>
LST_day_mean        3.4976  0.0000   -1.9369   21.0597
lu_industrial_area  0.5642  0.0000   -0.0000    0.0000
cropland_a          0.5449  0.0000   -1.6741    2.3852
water_bod_a         0.3887  0.0000   -0.7366    1.4454
non_built_area      0.1529  0.0000   -4.1205    2.0252
road_tertiary_len  -0.1477  0.0000   -0.0000    0.0000
built_up_a          0.1431  0.0000   -0.6631    0.9639
wetland_a           0.1165  0.0000   -0.1189    0.3125
pop_sum_m           0.0527  0.0000   -1.2763    0.8835
grassland_a         0.0415  0.0000   -0.4009    0.6188


##### LightGBM experimentation

In [73]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

warnings.filterwarnings('ignore')

# 1) Prepare raw‐unit dataset
X_lgb = baghdad_df[features_full].copy()
y_lgb = baghdad_df['TCI'].copy()

X_tr_lgb, X_te_lgb, y_tr_lgb, y_te_lgb = train_test_split(
    X_lgb, y_lgb, test_size=0.2, random_state=42
)

# 2) Parameter distributions for RandomizedSearch
param_dist = {
    'num_leaves':        [31, 50, 100],
    'max_depth':         [-1, 10, 20],
    'learning_rate':     [0.01, 0.05, 0.1],
    'n_estimators':      [100, 200, 500],
    'min_child_samples': [10, 20, 50],
    'subsample':         [0.6, 0.8, 1.0],
    'colsample_bytree':  [0.6, 0.8, 1.0]
}

lgb_reg = lgb.LGBMRegressor(random_state=42, n_jobs=-1)

rs_lgb = RandomizedSearchCV(
    estimator=lgb_reg,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring='r2',
    return_train_score=True,
    random_state=42,
    n_jobs=-1
)
rs_lgb.fit(X_tr_lgb, y_tr_lgb)

best_params_lgb = rs_lgb.best_params_

# 3) Refit best model on full training set
best_lgb = lgb.LGBMRegressor(**best_params_lgb, random_state=42)
best_lgb.fit(X_tr_lgb, y_tr_lgb)

# 4) Evaluate on train and test
for label, X_e, y_e in [('TRAIN', X_tr_lgb, y_tr_lgb), ('TEST', X_te_lgb, y_te_lgb)]:
    y_pred = best_lgb.predict(X_e)
    rmse   = np.sqrt(mean_squared_error(y_e, y_pred))
    r2     = r2_score(y_e, y_pred)
    print(f"LightGBM ({label}) → RMSE = {rmse:.4f}, R² = {r2:.4f}")

print("\nBest LightGBM parameters:")
print(best_params_lgb)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000432 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1752
[LightGBM] [Info] Number of data points in the train set: 403, number of used features: 24
[LightGBM] [Info] Start training from score 11221479.322727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [74]:
# Predictions
y_tr_pred = best_lgb.predict(X_tr_lgb)
y_te_pred = best_lgb.predict(X_te_lgb)

# Compute metrics
metrics = pd.DataFrame({
    'RMSE': [
        np.sqrt(mean_squared_error(y_tr_lgb, y_tr_pred)),
        np.sqrt(mean_squared_error(y_te_lgb, y_te_pred))
    ],
    'R2': [
        r2_score(y_tr_lgb, y_tr_pred),
        r2_score(y_te_lgb, y_te_pred)
    ]
}, index=['TRAIN', 'TEST'])

# Display nicely
print("LightGBM Performance Metrics")
print(metrics.to_string(float_format='%.4f'))

LightGBM Performance Metrics
              RMSE     R2
TRAIN 6849906.7318 0.8371
TEST  9608374.3596 0.8065


In [ ]:
# 1) Global SHAP importances
explainer = shap.TreeExplainer(best_lgb)
shap_vals  = explainer.shap_values(X_te_lgb)
# mean absolute impact on model output
shap_imp   = pd.Series(np.abs(shap_vals).mean(axis=0),
                       index=X_te_lgb.columns,
                       name='mean|SHAP|').sort_values(ascending=False)

print("Top-10 global drivers by SHAP:")
print(shap_imp.head(10).to_string(float_format='%.4f'))

# Optional: visual summary
# shap.summary_plot(shap_vals, X_te_lgb, plot_type="bar")

# 2) Elasticity approximation (finite-difference)
def elasticity_pct(model, X, feature, delta=0.01):
    """
    Approximates ε = (%Δy) / (%Δx) for a raw-unit model:
      For each row i: bump xi by xi*delta, compute new y,
      then (Δy / y) / delta.
    Returns vector of local elasticities for each observation.
    """
    x_base = X[feature].values
    bump   = np.where(x_base!=0, x_base*delta, delta)
    X_up   = X.copy()
    X_up[feature] = x_base + bump

    y0 = model.predict(X)
    y1 = model.predict(X_up)
    # avoid /0
    with np.errstate(divide='ignore', invalid='ignore'):
        eps = ((y1 - y0) / y0) / delta
    return eps

# 3) Compute elasticities for each feature across the test set
elas_dict = {}
for feat in X_te_lgb.columns:
    eps = elasticity_pct(best_lgb, X_te_lgb, feat, delta=0.01)
    # summarise: mean, median, 10% & 90% deciles
    elas_dict[feat] = [
        np.nanmean(eps),
        np.nanmedian(eps),
        np.nanpercentile(eps, 10),
        np.nanpercentile(eps, 90)
    ]

elas_df = pd.DataFrame.from_dict(
    elas_dict,
    orient='index',
    columns=['mean_elas','median_elas','q10','q90']
).sort_values('mean_elas', key=lambda s: s.abs(), ascending=False)

print("\nElasticity summary (percent-response per 1% feature bump):")
print(elas_df.head(10).to_string(float_format='%.4f'))

Top-10 global drivers by SHAP:
road_primary_len      4618026.5990
poi_count             3937523.5479
LST_day_mean          2114417.4322
road_motorway_len     1763763.9300
non_built_area        1404821.6684
pop_sum_m             1159875.8743
lu_residential_area   1131794.4888
water_bod_a           1121480.6243
road_secondary_len    1039664.0811
lu_commercial_area     890669.2143

Elasticity summary (percent-response per 1% feature bump):
                    mean_elas  median_elas     q10    q90
LST_day_mean           1.1533       0.0000 -3.8567 7.1198
grassland_a           -0.9425       0.0000  0.0000 0.0000
road_primary_len      -0.6787       0.0000  0.0000 0.0000
water_bod_a           -0.6063       0.0000  0.0000 0.0000
cropland_a            -0.3080       0.0000  0.0000 0.0000
built_up_a            -0.2184       0.0000  0.0000 0.0000
non_built_area        -0.1665       0.0000  0.0000 0.0000
lu_industrial_area    -0.1052       0.0000  0.0000 0.0000
NTL_mean              -0.1023       0

##### Neural Network

In [80]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

# 1) Prepare & scale data
X = baghdad_df[features_full].values
y = baghdad_df['TCI'].values.reshape(-1,1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_tr, X_tmp, y_tr, y_tmp = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)
X_val, X_te, y_val, y_te = train_test_split(
    X_tmp, y_tmp, test_size=0.5, random_state=42)

# 2) DataLoaders
def make_loader(X, y, bs, shuffle=False):
    ds = TensorDataset(torch.from_numpy(X).float(),
                       torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=bs, shuffle=shuffle)

# 3) Model with BatchNorm
class MLP(nn.Module):
    def __init__(self, dims, dropout):
        super().__init__()
        layers = []
        for i in range(len(dims)-1):
            layers += [
                nn.Linear(dims[i], dims[i+1]),
                nn.BatchNorm1d(dims[i+1]),
                nn.ReLU(),
                nn.Dropout(dropout)
            ]
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

# 4) Training routine with early stopping
def train_nn(hidden_dims=[32,16], dropout=0.2, lr=1e-3, wd=1e-4,
             batch_size=64, epochs=100, patience=10):
    dims = [X_tr.shape[1]] + hidden_dims + [1]
    model = MLP(dims, dropout).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=10)
    loss_fn = nn.MSELoss()

    tr_load = make_loader(X_tr, y_tr, batch_size, shuffle=True)
    val_load= make_loader(X_val, y_val, batch_size)

    best_val_loss, wait = np.inf, 0
    for ep in range(epochs):
        # train
        model.train()
        for xb,yb in tr_load:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss_fn(model(xb), yb).backward()
            opt.step()
            sched.step()

        # validate
        model.eval()
        preds, truths = [], []
        with torch.no_grad():
            for xb,yb in val_load:
                out = model(xb.to(device)).cpu().numpy()
                preds.append(out); truths.append(yb.numpy())
        val_loss = mean_squared_error(
            np.vstack(truths), np.vstack(preds))
        if val_loss < best_val_loss:
            best_val_loss, wait = val_loss, 0
            best_weights = model.state_dict()
        else:
            wait += 1
            if wait >= patience:
                break

    # load best
    model.load_state_dict(best_weights)
    # evaluate on train/val/test
    def eval_set(Xs, ys):
        yhat = model(torch.from_numpy(Xs).float().to(device)).cpu().detach().numpy()
        return (
            np.sqrt(mean_squared_error(ys, yhat)),
            r2_score(ys, yhat)
        )

    rmse_tr, r2_tr = eval_set(X_tr, y_tr)
    rmse_val, r2_val = eval_set(X_val, y_val)
    rmse_te, r2_te = eval_set(X_te, y_te)

    return {
      'model': model,
      'rmse_tr': rmse_tr, 'r2_tr': r2_tr,
      'rmse_val': rmse_val, 'r2_val': r2_val,
      'rmse_te': rmse_te, 'r2_te': r2_te
    }

# 5) Quick grid
configs = [
    {'hidden_dims':[32,16], 'dropout':0.1, 'lr':1e-3},
    {'hidden_dims':[64,32], 'dropout':0.2, 'lr':5e-4},
]
best = None
for cfg in configs:
    res = train_nn(**cfg)
    print(cfg, res['r2_val'], res['r2_te'])
    if best is None or res['r2_te']>best['r2_te']:
        best = res

print("\nBest NN Performance:")
print(f"Train R²={best['r2_tr']:.4f}, Val R²={best['r2_val']:.4f}, Test R²={best['r2_te']:.4f}")


{'hidden_dims': [32, 16], 'dropout': 0.1, 'lr': 0.001} -0.46893307931640615 -0.522192213517753
{'hidden_dims': [64, 32], 'dropout': 0.2, 'lr': 0.0005} -0.46893307363275527 -0.5221922030759059

Best NN Performance:
Train R²=-0.4244, Val R²=-0.4689, Test R²=-0.5222
